In [ ]:
import sys
sys.path.append("../")
import sense.utils.model_loading as ml
import advertorch.attacks as adver
import torchvision.transforms as T
from torchvision import datasets
import torch
import torch.nn as nn
from importlib import reload

import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np



reload(adver)

In [ ]:
torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)

In [ ]:
! mkdir -p ./result/PGD_conv_check

In [ ]:
#config: 
n_restart=20
nb_iter = 500
batch_size=100

# CIFAR

In [ ]:
model = ml.model_loader("SENSE","../trained_models")

In [ ]:
transform_test = T.Compose([T.ToTensor()])

test_dataset = datasets.CIFAR10("../../../../data", train=False, transform=transform_test)

loader =  torch.utils.data.DataLoader(test_dataset, batch_size = batch_size)

In [ ]:
PGD = adver.LinfPGDAttack_loss_track(model, eps=8./255, eps_iter=2./255, nb_iter=nb_iter)

In [ ]:
x,y = iter(loader).next()
x,y = x.cuda(), y.cuda()

In [ ]:
worst_fooled = torch.zeros((nb_iter,y.shape[0]),dtype=bool)
fooled_list=[]
losses_list=[]
for r in range(n_restart):
    
    print(F"iteration:{r}\n")
    
    _, losses,fooled = PGD(x,y)
    
    losses = losses.cpu()
    fooled = fooled.cpu()
    losses_list.append(losses)
    fooled_list.append(fooled)
    worst_fooled +=fooled

In [ ]:
#torch.save(losses_list, "./result/PGD_conv_check/cifiar_losses_list.dat")
#torch.save(worst_fooled, "./result/PGD_conv_check/cifiar_worst_fooled.dat")
#torch.save(fooled_list, "./result/PGD_conv_check/cifiar_fooled_list.dat")

In [ ]:
#losses_list = torch.load("./result/PGD_conv_check/cifiar_losses_list.dat")
#worst_fooled = torch.load("./result/PGD_conv_check/cifiar_worst_fooled.dat")
#fooled_list = torch.load("./result/PGD_conv_check/cifiar_fooled_list.dat")

In [ ]:
cum_worst_fooled = [fooled_list[0]]
for i in range(1,len(fooled_list)):
    cum_worst_fooled.append(cum_worst_fooled[-1]+fooled_list[i]) 
mean_cum_worst_fooled = [1-fooled.float().mean(dim=1).detach().view(1,-1) for fooled in cum_worst_fooled ]

In [ ]:
mean_cum_worst_fooled = torch.t(torch.cat(mean_cum_worst_fooled))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
#fig.suptitle('CIFAR with PGD for the first 100 examples\n\n')
ax1.set_title('PGD loss')
ax2.set_title('Accuracy, \nWorst among 20 random restarts')
ax1.set_xlabel('PGD steps')
ax2.set_xlabel('PGD steps')

x = np.array(range(nb_iter))

ax1.plot(x,losses_list[0].detach().cpu().numpy())
ax2.plot(x,mean_cum_worst_fooled.numpy())

# MNIST

In [ ]:
#configs: 
nb_iter = 1000
batch_size=100

In [ ]:
from sense.others.CNNs import SmallCNN

model = SmallCNN()    
model = nn.DataParallel(model).cuda()
checkpoint = torch.load("../appendix/mnist/trained_models/mnist/SENSE_checkpoint500.dict")
model.load_state_dict(checkpoint['state_dict'])
model.eval()

In [ ]:
transform_test = T.Compose([T.ToTensor()])

test_dataset = datasets.MNIST("../../../../data", train=False, transform=transform_test)

loader =  torch.utils.data.DataLoader(test_dataset, batch_size = batch_size)

In [ ]:
PGD = adver.LinfPGDAttack_loss_track(model, eps=0.3, eps_iter=0.01, nb_iter=nb_iter)
x,y = iter(loader).next()
x,y = x.cuda(), y.cuda()

In [ ]:
worst_fooled = torch.zeros((nb_iter,y.shape[0]),dtype=bool)
fooled_list=[]
losses_list=[]
for r in range(n_restart):
    
    print(F"iteration:{r}\n")
    
    _, losses,fooled = PGD(x,y)
    
    losses = losses.cpu()
    fooled = fooled.cpu()
    losses_list.append(losses)
    fooled_list.append(fooled)
    worst_fooled +=fooled

In [ ]:
#torch.save(losses_list, "./result/PGD_conv_check/mnist_losses_list.dat")
#torch.save(worst_fooled, "./result/PGD_conv_check/mnist_worst_fooled.dat")
#torch.save(fooled_list, "./result/PGD_conv_check/mnist_fooled_list.dat")

In [ ]:
#losses_list = torch.load("./result/PGD_conv_check/mnist_losses_list.dat")
#worst_fooled = torch.load("./result/PGD_conv_check/mnist_worst_fooled.dat")
#fooled_list = torch.load("./result/PGD_conv_check/mnist_fooled_list.dat")

In [ ]:
cum_worst_fooled = [fooled_list[0]]
for i in range(1,len(fooled_list)):
    cum_worst_fooled.append(cum_worst_fooled[-1]+fooled_list[i]) 
mean_cum_worst_fooled = [1-fooled.float().mean(dim=1).detach().view(1,-1) for fooled in cum_worst_fooled ]

In [ ]:
mean_cum_worst_fooled = torch.t(torch.cat(mean_cum_worst_fooled))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('MNIST with PGD for the first 100 examples')
ax1.set_title('PGD loss')
ax2.set_title('Accuracy, Worst among 20 random restarts')
ax1.set_xlabel('PGD steps')
ax2.set_xlabel('PGD steps')

#1 : epsilon = 0.3
x = np.array(range(nb_iter))

ax1.plot(x,losses_list[0].detach().cpu().numpy())
ax2.plot(x,mean_cum_worst_fooled.numpy())